In [ ]:
import os
import time

import re
import copy
import numpy as np
import pandas as pd
import h5py
import tables
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import rioxarray as rxr
import math
import pickle 
import sklearn
import graphviz
import xgboost as xgb
import lightgbm as lgbm
from pickle import dump

import contextily as cx
import rasterio
import geopandas as gpd
from shapely.geometry import Point
#from mpl_toolkits.basemap import Basemap

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from xgboost import cv
from xgboost import XGBRegressor
from xgboost import plot_importance as plot_importance_XGB
from lightgbm import LGBMRegressor
from lightgbm import plot_importance as plot_importance_LGBM

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, Activation, Dropout
from os import listdir
from os.path import isfile, join
from platform import python_version

import re

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]


print(pd.__version__) # should be 1.3.0
print(sklearn.__version__) # should be 0.24.1
print(tf.__version__) # should be 2.4.0
print(python_version()) #should be 3.8.2

In [ ]:
#make Region identifier. The data already includes Region, but too many 'other' labels

def Region_id(df):
    
    for i in tqdm(range(0, len(df))):

        #Sierras
        #Northern Sierras
        if -122.5 <= df['Long'][i] <=-119 and 39 <=df['Lat'][i] <= 42:
            loc = 'N_Sierras'
            df['Region'].iloc[i] = loc

        #Southern Sierras
        if -121.2 <= df['Long'][i] <=-117 and 35 <=df['Lat'][i] <= 39:
            loc = 'S_Sierras'
            df['Region'].iloc[i] = loc




        #West Coast    
        #CACoastal (Ca-Or boarder)
        if df['Long'][i] <=-122.5 and df['Lat'][i] <= 42:
            loc = 'Ca_Coast'
            df['Region'].iloc[i] = loc



        #Oregon Coastal (Or)?
        if df['Long'][i] <=-122.7 and 42<= df['Lat'][i] <= 46:
            loc = 'Or_Coast'
            df['Region'].iloc[i] = loc



        #Olympis Coastal (Wa)
        if df['Long'][i] <=-122.5 and 46<= df['Lat'][i]:
            loc = 'Wa_Coast'
            df['Region'].iloc[i] = loc    



        #Cascades    
         #Northern Cascades
        if -122.5 <= df['Long'][i] <=-119.4 and 46 <=df['Lat'][i]:
            loc = 'N_Cascade'
            df['Region'].iloc[i] = loc



        #Southern Cascades
        if -122.7 <= df['Long'][i] <=-121 and 42 <=df['Lat'][i] <= 46:
            loc = 'S_Cascade'
            df['Region'].iloc[i] = loc



        #Eastern Cascades and Northern Idaho and Western Montana
        if -119.4 <= df['Long'][i] <=-116.4 and 46 <=df['Lat'][i]:
            loc = 'E_WA_N_Id_W_Mont'
            df['Region'].iloc[i] = loc
        #Eastern Cascades and Northern Idaho and Western Montana
        if -116.4 <= df['Long'][i] <=-114.1 and 46.6 <=df['Lat'][i]:
            loc = 'E_WA_N_Id_W_Mont'
            df['Region'].iloc[i] = loc






        #Eastern Oregon
        if -121 <= df['Long'][i] <=-116.4 and 43.5 <=df['Lat'][i] <= 46:
            loc = 'E_Or'
            df['Region'].iloc[i] = loc



        #Great Basin
        if -121 <= df['Long'][i] <=-112 and 42 <=df['Lat'][i] <= 43.5:
            loc = 'GBasin'
            df['Region'].iloc[i] = loc

        if -119 <= df['Long'][i] <=-112 and 39 <=df['Lat'][i] <= 42:
            loc = 'GBasin'
            df['Region'].iloc[i] = loc
            #note this section includes mojave too
        if -117 <= df['Long'][i] <=-113.2 and df['Lat'][i] <= 39:
            loc = 'GBasin'
            df['Region'].iloc[i] = loc


        #SW Mtns (Az and Nm)
        if -113.2 <= df['Long'][i] <=-107 and df['Lat'][i] <= 37:
            loc = 'SW_Mtns'
            df['Region'].iloc[i] = loc




        #Southern Wasatch + Utah Desert Peaks
        if -113.2 <= df['Long'][i] <=-109 and 37 <= df['Lat'][i] <= 39:
            loc = 'S_Wasatch'
            df['Region'].iloc[i] = loc
        #Southern Wasatch + Utah Desert Peaks
        if -112 <= df['Long'][i] <=-109 and 39 <= df['Lat'][i] <= 40:
            loc = 'S_Wasatch'
            df['Region'].iloc[i] = loc



        #Northern Wasatch + Bear River Drainage
        if -112 <= df['Long'][i] <=-109 and 40 <= df['Lat'][i] <= 42.5:
            loc = 'N_Wasatch'
            df['Region'].iloc[i] = loc




        #YellowStone, Winds, Big horns
        if -111 <= df['Long'][i] <=-106.5 and 42.5 <= df['Lat'][i] <= 45.8:
            loc = 'Greater_Yellowstone'
            df['Region'].iloc[i] = loc



        #North of YellowStone to Boarder
        if -112.5 <= df['Long'][i] <=-106.5 and 45.8 <= df['Lat'][i]:
            loc = 'N_Yellowstone'
            df['Region'].iloc[i] = loc



         #SW Montana and nearby Idaho
        if -112 <= df['Long'][i] <=-111 and 42.5 <= df['Lat'][i] <=45.8:
            loc = 'SW_Mont'
            df['Region'].iloc[i] = loc 
         #SW Montana and nearby Idaho
        if -113 <= df['Long'][i] <=-112 and 43.5 <= df['Lat'][i] <=45.8:
            loc = 'SW_Mont'
            df['Region'].iloc[i] = loc
        #SW Montana and nearby Idaho
        if -113 <= df['Long'][i] <=-112.5 and 45.8 <= df['Lat'][i] <=46.6:
            loc = 'SW_Mont'
            df['Region'].iloc[i] = loc



         #Sawtooths, Idaho
        if -116.4 <= df['Long'][i] <=-113 and 43.5 <= df['Lat'][i] <=46.6:
            loc = 'Sawtooth'
            df['Region'].iloc[i] = loc




        #Greater Glacier
        if -114.1 <= df['Long'][i] <=-112.5 and 46.6 <= df['Lat'][i]:
            loc = 'Greater_Glacier'
            df['Region'].iloc[i] = loc 



         #Southern Wyoming 
        if -109 <= df['Long'][i] <=-104.5 and 40.99 <= df['Lat'][i] <= 42.5 :
            loc = 'S_Wyoming'
            df['Region'].iloc[i] = loc 
        #Southern Wyoming
        if -106.5 <= df['Long'][i] <=-104.5 and 42.5 <= df['Lat'][i] <= 43.2:
            loc = 'S_Wyoming'
            df['Region'].iloc[i] = loc 




         #Northern Colorado Rockies
        if -109 <= df['Long'][i] <=-104.5 and 38.3 <= df['Lat'][i] <= 40.99:
            loc = 'N_Co_Rockies'
            df['Region'].iloc[i] = loc 



         #SW Colorado Rockies
        if -109 <= df['Long'][i] <=-106 and 36.99 <= df['Lat'][i] <= 38.3:
            loc = 'SW_Co_Rockies'
            df['Region'].iloc[i] = loc 




        #SE Colorado Rockies + Northern New Mexico
        if -106 <= df['Long'][i] <=-104.5 and 34 <= df['Lat'][i] <= 38.3:
            loc = 'SE_Co_Rockies'
            df['Region'].iloc[i] = loc  
        #SE Colorado Rockies + Northern New Mexico
        if -107 <= df['Long'][i] <=-106 and 34 <= df['Lat'][i] <= 36.99:
            loc = 'SE_Co_Rockies'
            df['Region'].iloc[i] = loc 
    

In [ ]:
def week_num(df):
        #week of water year
    weeklist = []

    for i in tqdm(range(0,len(df))):
        if df['Date'][i].month<11:
            y = df['Date'][i].year-1
        else:
            y = df['Date'][i].year
            
        WY_start = pd.to_datetime(str(y)+'-10-01')
        deltaday = df['Date'][i]-WY_start
        deltaweek = round(deltaday.days/7)
        weeklist.append(deltaweek)


    df['WYWeek'] = weeklist


In [ ]:
def NaReplacement(region, RegionSnotel, prev_RegionSnotel):
    
    #Make NA values mean snowpack values
    meanSWE = np.mean(RegionSnotel[region]['SWE'][RegionSnotel[region]['SWE']>0])
    RegionSnotel[region]['SWE'][RegionSnotel[region]['SWE']<0]= meanSWE
    
    
    prev_meanSWE = np.mean(prev_RegionSnotel[region]['SWE'][prev_RegionSnotel[region]['SWE']>0])
    prev_RegionSnotel[region]['SWE'][prev_RegionSnotel[region]['SWE']<0]= prev_meanSWE
    
    

    delta = RegionSnotel[region]['SWE']-prev_RegionSnotel[region]['SWE']
    delta = pd.DataFrame(delta)
    delta = delta.rename(columns = {'SWE':'Delta'})

    #get values that are not affected by NA
    delta = delta[delta['Delta']> -9000]

    #Get mean Delta to adjust observed SWE
    meanD = np.mean(delta['Delta'])

    #go and fix current SWE observations
    #Get bad obsevations and snotel sites
    badSWE_df = RegionSnotel[region][RegionSnotel[region]['SWE']< 0].copy()
    bad_sites = list(badSWE_df.index)


    #remove bad observations from SWE obsevations
    RegionSnotel[region] = RegionSnotel[region][RegionSnotel[region]['SWE'] >= 0]

    #Fix bad observatoins by taking previous obs +/- mean delta SWE
 #   print('Fixing these bad sites in ', region, ':')
  #  for badsite in bad_sites:
   #     print(badsite)
    #    badSWE_df.loc[badsite,'SWE']=prev_RegionSnotel[region].loc[badsite]['SWE'] + meanD

    #Add observations back to DF
    RegionSnotel[region] = pd.concat([RegionSnotel[region], badSWE_df])
    
    return  RegionSnotel[region]


In [ ]:
#set up model prediction function
def SWE_Initial_Predict(Forecast, Region, Region_optfeatures):
    
    #region specific features
    features = Region_optfeatures[Region]
    
    #Make prediction dataframe
    forecast_data = Forecast[Region].copy()
    forecast_data = forecast_data[features]
   
    
    #change all na values to prevent scaling issues
    forecast_data[forecast_data< -9000]= -10

    
    #load and scale data
    
    #set up filepath to extract best model
    checkpoint_filepath = 'Model/Initial_Models_Final/'+Region+ '/'
    model = checkpoint_filepath+Region+'_model.h5'
    print(model)
    model=load_model(model)
    
    
    #load SWE scaler
    SWEmax = np.load(checkpoint_filepath+Region+'_SWEmax.npy')
    SWEmax = SWEmax.item()
    
    #load features scaler
    #save scaler data here too
    scaler =  pickle.load(open(checkpoint_filepath+Region+'_scaler.pkl', 'rb'))
    scaled = scaler.transform(forecast_data)
    x_forecast = pd.DataFrame(scaled, columns = forecast_data.columns)
   
    
    
     #make predictions and rescale
    y_forecast = (model.predict(x_forecast))

    y_forecast[y_forecast < 0 ] = 0
    y_forecast = (SWEmax * y_forecast)
    Forecast[Region][datecol] = y_forecast
    
    
    #plot predictions    
    plt.scatter( Forecast[Region]['elevation_m'],Forecast[Region][datecol], s=5, color="blue", label="Predictions")
    plt.xlabel('elevation m')
    plt.ylabel('Predicted SWE')
    plt.legend()
    
    
    #plt.plot(x_ax, y_pred, lw=0.8, color="red", label="predicted")
    plt.title(Region)
    plt.show()
    
    
    #plot geolocation information
    _geom = [Point(xy) for xy in zip(Forecast[Region]['Long'], Forecast[Region]['Lat'])]
    _geom_df = gpd.GeoDataFrame(Forecast[Region], crs="EPSG:4326", geometry=_geom)

    dfmax = max(Forecast[Region][datecol])*1.05

    # fig, ax = plt.subplots(figsize=(14,6))
    ax = _geom_df.plot(datecol, cmap="cool", markersize=30,figsize=(25,25), legend=True, vmin=0, vmax=dfmax)#vmax=test_preds['delta'].max(), vmin=test_preds['delta'].min())
    cx.add_basemap(ax, alpha = .7, crs=_geom_df.crs.to_string())


    plt.show()
    
    
    
    return Forecast[Region]

In [ ]:
os.getcwd()
os.chdir('..')
os.getcwd()

In [ ]:
'''
Get Up to date snotel measurements and connect to validation DF/Regions

This data assimilation codes makes the second week's observations
and saves them to build on for the next set of observations (week 3-)

'''

In [ ]:
#Set up training DF with key metadata per site
#All coordinates of 1 km polygon used to develop ave elevation, ave slope, ave aspect
path = 'Data/Processed/RegionVal.pkl'
#load regionalized geospatial data
RegionTest = open(path, "rb")
RegionTest = pickle.load(RegionTest)


#input current and previous weeks dates (these upload csv, must match dates)
date = '01_20_2022'
datekey = '1/20/2022'

prevdate = '01_13_2022'
prevdatekey = '1/13/2022'

#load ground truth values (SNOTEL): Testing
obs_path = 'Data/Pre_Processed/ground_measures_features_' + date + '.csv'
GM_Test = pd.read_csv(obs_path)

#change first column to station id
GM_Test = GM_Test.rename(columns = {'Unnamed: 0':'station_id'})


#Fill NA observations
GM_Test[datekey] = GM_Test[datekey].fillna(-9999)

#drop na and put into modeling df format
GM_Test = GM_Test.melt(id_vars=["station_id"]).dropna()

#change variable to Date and value to SWE
GM_Test = GM_Test.rename(columns ={'variable': 'Date', 'value':'SWE'})

#load ground truth meta
GM_Meta = pd.read_csv('Data/Pre_Processed/ground_measures_metadata.csv')

#merge testing ground truth location metadata with snotel data
GM_Test = GM_Meta.merge(GM_Test, how='inner', on='station_id')
GM_Test = GM_Test.set_index('station_id')
GM_Test.rename(columns={'name': 'location', 'latitude': 'Lat', 'longitude': 'Long', 'value': 'SWE'}, inplace=True)




#load ground truth values (SNOTEL): previous week
obs_path = 'Data/Pre_Processed/ground_measures_features_' + prevdate + '.csv'
GM_Prev = pd.read_csv(obs_path)

#change first column to station id
GM_Prev = GM_Prev.rename(columns = {'Unnamed: 0':'station_id'})

#Fill NA observations
GM_Prev[prevdatekey] = GM_Prev[prevdatekey].fillna(-9999)

#drop na and put into modeling df format
GM_Prev = GM_Prev.melt(id_vars=["station_id"]).dropna()

#change variable to Date and value to SWE
GM_Prev = GM_Prev.rename(columns ={'variable': 'Date', 'value':'SWE'})

#merge testing ground truth location metadata with snotel data
GM_Prev = GM_Meta.merge(GM_Prev, how='inner', on='station_id')
GM_Prev = GM_Prev.set_index('station_id')
GM_Prev.rename(columns={'name': 'location', 'latitude': 'Lat', 'longitude': 'Long', 'value': 'SWE'}, inplace=True)


In [ ]:
#Make a dictionary for current snotel observations
Snotel = GM_Test.copy()
Snotel['Region'] = 'other'
Region_id(Snotel)
RegionSnotel  = {name: Snotel.loc[Snotel['Region'] == name] for name in Snotel.Region.unique()}

#Make a dictionary for previous week's snotel observations
prev_Snotel = GM_Prev.copy()
prev_Snotel['Region'] = 'other'
Region_id(prev_Snotel)
prev_RegionSnotel  = {name: prev_Snotel.loc[prev_Snotel['Region'] == name] for name in prev_Snotel.Region.unique()}



#add week number to observations
for i in RegionTest.keys():
    RegionTest[i] = RegionTest[i].reset_index(drop=True)
    RegionTest[i]['Date'] = pd.to_datetime(RegionSnotel[i]['Date'][0])
    week_num(RegionTest[i])

In [ ]:
##checking to make sure not bad features are used as key model features
OG_Region_list = ['N_Sierras',
               'S_Sierras',
               'Greater_Yellowstone',
               'N_Co_Rockies',
               'SW_Mont',
               'SW_Co_Rockies',
               'GBasin',
               'N_Wasatch',
               'N_Cascade',
               'S_Wasatch',
               'SW_Mtns',
               'E_WA_N_Id_W_Mont',
               'S_Wyoming',
               'SE_Co_Rockies',
               'Sawtooth',
               'Ca_Coast',
               'E_Or',
               'N_Yellowstone',
               'S_Cascade',
               'Wa_Coast',
               'Greater_Glacier',
               'Or_Coast'
              ]

#set up dataframe to save to be future GM_Pred
col = list(GM_Test.columns)+['Region']
Future_GM_Pred = pd.DataFrame(columns = col)

for region in OG_Region_list:
    RegionSnotel[region] = NaReplacement(region, RegionSnotel, prev_RegionSnotel)
    RegionSnotel[region]['Prev_SWE'] =prev_RegionSnotel[region]['SWE']
    RegionSnotel[region]['Delta_SWE'] = RegionSnotel[region]['SWE'] - RegionSnotel[region]['Prev_SWE']
    
    #make dataframe to function as next forecasts GM_Prev
    Future_GM_Pred = Future_GM_Pred.append(RegionSnotel[region])
    
#Need to save 'updated non-na' df's
GM_path = 'Data/Processed/DA_ground_measures_features_' + date + '.csv'

Future_GM_Pred.to_csv(GM_path)

#This needs to be here to run in next codeblock
Regions = list(RegionTest.keys()).copy()

In [ ]:

#Make dictionary in Regions dict for each region's dictionary of Snotel sites
#Regions = list(RegionTrain.keys()).copy()

for i in tqdm(Regions):
    
    snotel = i+'_Snotel'
    RegionTest[snotel] = {site: RegionSnotel[i].loc[site] for site in RegionSnotel[i].index.unique()}
    
   #get training and testing sites that are the same
    test = RegionTest[snotel].keys()
    
    for j in test:
        RegionTest[snotel][j] = RegionTest[snotel][j].to_frame().T
    #remove items we do not need
        RegionTest[snotel][j] = RegionTest[snotel][j].drop(columns = ['Long', 'Lat', 'location',
                                                                      'elevation_m', 'state', 'Region'])
    #make date index
        RegionTest[snotel][j] = RegionTest[snotel][j].set_index('Date')
        
    #rename columns to represent site info
        colnames = RegionTest[snotel][j].columns
        sitecolnames = [x +'_'+ j for x in colnames]
        names = dict(zip(colnames, sitecolnames))
        RegionTest[snotel][j] = RegionTest[snotel][j].rename(columns = names)
        
        

In [ ]:
#make a df for training each region, 

for R in tqdm(Regions):
    snotels = R+'_Snotel'  
   # RegionTest[R] = RegionTest[R].reset_index()
   # print(R)
    sites = list(RegionTest[R]['cell_id'])
    sitelen = len(sites)-1
    RegionTest[R] = RegionTest[R].set_index('cell_id')
    
    
    for S in RegionTest[snotels].keys():
       # print(S)
        RegionTest[snotels][S] = RegionTest[snotels][S].append([RegionTest[snotels][S]]*sitelen, ignore_index = True)
        RegionTest[snotels][S].index = sites
        RegionTest[R]= pd.concat([RegionTest[R], RegionTest[snotels][S].reindex(RegionTest[R].index)], axis=1)
    
    RegionTest[R] = RegionTest[R].fillna(-9999.99)
    del RegionTest[R]['Date']

In [ ]:
#Perform the splitting for S_Sierras High and Low elevations
RegionTest['S_Sierras_High'] =RegionTest['S_Sierras'].loc[RegionTest['S_Sierras']['elevation_m'] > 2500].copy()
RegionTest['S_Sierras_Low'] = RegionTest['S_Sierras'].loc[RegionTest['S_Sierras']['elevation_m'] <= 2500].copy()
del RegionTest['S_Sierras']

In [ ]:
#save dictionaries as pkl
# create a binary pickle file 
path = 'Data/Processed/ValidationDF_'+date + '.pkl'

RVal = open(path,"wb")


# write the python object (dict) to pickle file
pickle.dump(RegionTest,RVal)


# close file
RVal.close()


In [ ]:
'''
Take in initial conditions Observations for use in an initial SWE prediction
'''
#load first SWE observation forecasting dataset with prev and delta swe for observations. 

date = '01_20_2022'
datecol = '2022-01-20'


path = 'Data/Processed/ValidationDF_'+date + '.pkl'


#load regionalized forecast data
Forecast = open(path, "rb")

Forecast = pickle.load(Forecast)




#load RFE optimized features
Region_optfeatures= pickle.load(open("Model/Initial_Models_Final/opt_features_initial.pkl", "rb"))





In [ ]:


### define new regions
Region_list = ['N_Sierras',
               'S_Sierras_High',
               'S_Sierras_Low',
               'Greater_Yellowstone',
               'N_Co_Rockies',
               'SW_Mont',
               'SW_Co_Rockies',
               'GBasin',
               'N_Wasatch',
               'N_Cascade',
               'S_Wasatch',
               'SW_Mtns',
               'E_WA_N_Id_W_Mont',
               'S_Wyoming',
               'SE_Co_Rockies',
               'Sawtooth',
               'Ca_Coast',
               'E_Or',
               'N_Yellowstone',
               'S_Cascade',
               'Wa_Coast',
               'Greater_Glacier',
               'Or_Coast'
              ]
#Reorder regions
Forecast = {k: Forecast[k] for k in Region_list}


In [ ]:
#make and save predictions
Prev_df = pd.DataFrame()
predictions ={}
for Region in Region_list:
    print(Region)
    predictions[Region] = SWE_Initial_Predict(Forecast, Region, Region_optfeatures)
    predictions[Region] = pd.DataFrame(predictions[Region])
    
    del predictions[Region]['geometry']
    
    Prev_df = Prev_df.append(pd.DataFrame(predictions[Region][datecol]))
    Prev_df = pd.DataFrame(Prev_df)
    
    
    predictions[Region].to_hdf('Predictions/predictions'+datecol+'.h5', key = Region)
    
    
#load submission DF and add predictions
subdf = pd.read_csv('Predictions/submission_format.csv')
subdf = subdf.rename(columns = {'Unnamed: 0':'cell_id'})
subdf = subdf.set_index('cell_id')

sub_index = subdf.index
Prev_df = Prev_df.loc[sub_index]
subdf[datecol] = Prev_df[datecol].astype(float)
subdf['2022-01-13'] = subdf[datecol].astype(float)
subdf.to_csv('Predictions/submission_format_'+datecol+'.csv')